In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd

In [2]:
#selenium path to chromedriver
path = Service("C:\\Users\\siyun\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")

In [3]:
#scrapes headers from nfl combine website to be used as column names for dataframe
NFL_url = 'https://nflcombineresults.com/nflcombinedata.php?'

driver = webdriver.Chrome(service=path)
driver.get(NFL_url)

headers = driver.find_elements(By.TAG_NAME, "thead")
for element in headers:
    text = element.text

col_names = text.split("\n")
col_names = col_names[:-1] + col_names[-1].split(' ')
#col_names.remove('Wonderlic')

driver.quit()

In [4]:
#create data table to hold all scraped values
data_table = []

In [5]:
#scrape player combine data
for year in range(1987, 2024):
    
    NFL_url = 'https://nflcombineresults.com/nflcombinedata.php?year={yrs}&pos=&college='.format(yrs=year)
    driver = webdriver.Chrome(service=path)
    driver.get(NFL_url)

    table = driver.find_element(By.ID, 'datatable')
    rows = table.find_elements(By.TAG_NAME, 'tr')

    for row in rows[1:]:  # Skip the header row
        cols = row.find_elements(By.TAG_NAME, 'td')
        player_data = [col.text if col.text.strip() != '' else '0' for col in cols]
        data_table.append(player_data)
    driver.quit()

In [13]:
#transform data table into dataframe
df_all_players = pd.DataFrame(data_table, columns=col_names)
#export dataframe to csv file to checkpoint progress
df_all_players.to_csv('csv_files//combine_dataset.csv', index=False)

In [14]:
#create data table to hold all pro bowl player data
all_pro_players = []

In [15]:
#scrape pro bowl player data
for year in range(1987, 2023):

    NFL_url ='https://www.pro-football-reference.com/years/{yrs}/probowl.htm'.format(yrs=year)
    driver = webdriver.Chrome(service=path)
    driver.get(NFL_url)

    table = driver.find_element(By.ID, 'pro_bowl')
    rows = table.find_elements(By.TAG_NAME, 'tr')

    for row in rows[1:]:  # Skip the header row
        cols = row.find_elements(By.TAG_NAME, 'td')
        player_data = [col.text if col.text.strip() != '' else '0' for col in cols]
        player_data.append(year)
        all_pro_players.append(player_data)
    driver.quit()

In [26]:
print(all_pro_players.__str__())

[['Fredd Young', 'AFC', 'SEA', '26', '3', '13', '13', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '9.0', '1', 'PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm', 1987], ['Jim Arnold', 'NFC', 'DET', '26', '4', '11', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0.0', '0', 'PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm', 1987], ['Joey Browner', 'NFC', 'MIN', '27', '4', '12', '12', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1.0', '6', 'PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm', 1987], ['Reggie White', 'NFC', 'PHI', '26', '2', '12', '12', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '76', '21.0', '0', 'PFW: 1st Tm All-Conf., UPI: 1st Tm All-Conf., AP: 1st Tm, FW: 1st Tm, NEA: 1st Tm, PFW: 1st Tm, SN: 1st Tm', 1987], ['Morten Andersen', 'NFC', 'N

In [37]:
temp_list = []
for inner_list in all_pro_players:
    if (len(inner_list)>2):
        temp_list.append([inner_list[0], inner_list[3], inner_list[-1]])

In [41]:
print(temp_list[-1])

['Justin Hardee', '28', 2022]
